In [ ]:
import sys
sys.path.insert(0, "../")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
from pathlib import Path
import tifffile
import gc

In [ ]:
INPUT_PATH = Path("/hdd/kaggle/hubmap/input_v2")   
df_train = pd.read_csv(INPUT_PATH / "train.csv")
train_info = pd.read_csv(INPUT_PATH / "HuBMAP-20-dataset_information.csv")
train_tiffs = list((INPUT_PATH / "train").glob('*.tiff'))

In [ ]:
train_info.columns

In [ ]:
masks = list(Path("/hdd/kaggle/hubmap/input_v2/train_v3_4096_1024/masks").glob("*.png"))

In [ ]:
for i, img_path in enumerate(train_tiffs):
    image = tifffile.imread(img_path).squeeze()
    print(img_path,'\n', image.shape)
    if image.shape[0]==3:
        image = np.transpose(image, (1,2,0))
    h, w  = image.shape[:2]
    image = cv2.resize(image, (int(w//10), int(h//10)))
    
    mask = np.zeros((h, w), dtype=np.uint8)
    cur_masks = [(int(x.stem.split("_")[1]), int(x.stem.split("_")[2])) for x in masks if x.stem.split("_")[0] == Path(img_path).stem]
    for x, y in cur_masks:
        mask[y:y+2048*2, x:x+2048*2] = 1
    mask = cv2.resize(mask, (int(w//10), int(h//10)))   
    plt.figure(figsize=(10,10))
    plt.grid()
    plt.imshow(image)
    plt.imshow(mask, alpha=0.4)
    plt.show()
    del image
    gc.collect()


In [ ]:
image = tifffile.imread("/hdd/kaggle/hubmap/input_v2/train/aaa6a05cc.tiff").squeeze()
# print(img_path,'\n', image.shape)
if image.shape[0]==3:
    image = np.transpose(image, (1,2,0))
h, w  = image.shape[:2]
image = cv2.resize(image, (int(w//10), int(h//10)))
plt.figure(figsize=(30,30))
plt.grid()
plt.imshow(image)

In [ ]:
!ls ../../input/zarr_train_orig

In [ ]:
def _check_background(img, crop_size) -> bool:
    s_th = 40  # saturation blancking threshold
    p_th = 1000 * (crop_size // 256) ** 2  # threshold for the minimum number of pixels
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    _, ss, _ = cv2.split(hsv)
    background = False if (ss > s_th).sum() <= p_th or img.sum() <= p_th else True
    return background


def get_crop_coords(image, crop_size, step):
    h, w = image.shape[:2]
    coord = []
    for y in tqdm(range(0, h, step)):
        for x in range(0, w, step):

            if x + crop_size > w:
                x = w - crop_size
            if y + crop_size > h:
                y = h - crop_size
                
            crop_image = image[y: y+crop_size, x:x+crop_size]
            if _check_background(crop_image, crop_size):
                coord.append((x, y, crop_size))
                
    return np.array(coord)    
            
#             image[y: y+40, x:x+4096] = 255
#             image[y + 4096 - 40: y + 4096, x:x+4096] = 255
#             image[y: y + 4096, x: x + 40] = 255
#             image[y: y + 4096, x + 4096 - 40: x + 4096] = 255

In [ ]:
import zarr
zarr = zarr.open("../../input/zarr_train_orig", mode="r")
image = zarr['aa05346ff'][:]
h, w = image.shape[:2]

coords = get_crop_coords(image, 1024, 1024)

for x, y, crop_size in coords:
    image[y: y+40, x:x+crop_size] = 255
    image[y + crop_size - 40: y + crop_size, x:x+crop_size] = 255
    image[y: y + crop_size, x: x + 40] = 255
    image[y: y + crop_size, x + crop_size - 40: x + crop_size] = 255
# for y in range(0, h, 4096):
#     for x in range(0, w, 4096):
#         image[y: y+40, x:x+4096] = 255
#         image[y + 4096 - 40: y + 4096, x:x+4096] = 255
#         image[y: y + 4096, x: x + 40] = 255
#         image[y: y + 4096, x + 4096 - 40: x + 4096] = 255
            
image = cv2.resize(image, (int(w//10), int(h//10)))
plt.figure(figsize=(30,30))
plt.grid()
plt.imshow(image)

In [ ]:
x = [["one", 1, 4],["two", 2, 5],["three", 3, 6]]
xx = pd.DataFrame(x, columns=["img_id", "x", "y"])

In [ ]:
w, e, r = xx.loc[0]

In [ ]:
r

In [ ]:
!ls -l /hdd/kaggle/hubmap/input_v2/train_v1_1024

In [ ]:
df = pd.read_csv("/hdd/kaggle/hubmap/input_v2/train_v1_1024/meta.csv")

In [ ]:
img_ids = df['img_id'].unique()

for img_id in img_ids:
    glom_sq = df[df['img_id']==img_id]["glomerulus_pix"].sum()
    tis_sq = len(df[df['img_id']==img_id])*(1024**2)
    print(f"{img_id}: {glom_sq/tis_sq:.3f} sq: {glom_sq}")

In [ ]:
FOLD_IMGS = {
    0: ["4ef6695ce", "0486052bb", "2f6ecfcdf"],
    1: ["c68fe75ea", "095bf7a1f", "aaa6a05cc"],
    2: ["afa5e8098", "1e2425f28", "b2dc8411c"],
    3: ["cb2d976f4", "8242609fa", "54f2eec69"],
    4: ["26dc41664", "b9a3865fc", "e79de561c"],
}